In [1]:
# Import Libraries
import mlflow
import pandas as pd
import xgboost
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error


In [2]:
# defining a function to quickly read and prepare data
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['hour'] = df.lpep_pickup_datetime.dt.hour
    df['dayofweek'] = df.lpep_pickup_datetime.dt.day_of_week

    categorical = ['PULocationID', 'DOLocationID', 'hour', 'dayofweek', 'VendorID']
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    return df

In [3]:
# Reading and preparing the dataset
df_train = read_dataframe('/home/ubuntu/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/ubuntu/data/green_tripdata_2021-02.parquet')

In [4]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment-xgboost")

2022/05/25 11:39:16 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment-xgboost' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-experiment-xgboost', tags={}>

In [5]:
# defing feature types
categorical = ['PU_DO', 'hour', 'dayofweek', 'VendorID'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# applying one hot encoding
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [6]:
# defining the target
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
# defining xboost level data

training_data = xgboost.DMatrix(X_train, label=y_train)
valid = xgboost.DMatrix(X_val, label=y_val)

In [12]:
def objective(params):
    
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        model = xgboost.train(
            params = params,
            dtrain = training_data,
            num_boost_round = 100,
            evals = [(valid, "validation")],
            early_stopping_rounds = 20
        )

        y_pred = model.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -6, 1),
    'reg_lambda': hp.loguniform('reg_lambda', -5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective' : 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)



[12:34:44] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:10.74953                          
[1]	validation-rmse:7.58822                           
[2]	validation-rmse:6.75036                           
[3]	validation-rmse:6.50759                           
[4]	validation-rmse:6.41371                           
[5]	validation-rmse:6.38867                           
[6]	validation-rmse:6.37057                           
[7]	validation-rmse:6.36410                           
[8]	validation-rmse:6.35963                           
[9]	validation-rmse:6.35484                           
[10]	validation-rmse:6.35007                          
[11]	validation-rmse:6.34516                          
[12]	validation-rmse:6.34450                          
[13]	validation-rmse:6.33968                          
[14]	validation-rmse:6.33659                          
[15]	validation-rmse:6.33532                          
[

In [17]:
# Train the best model

with mlflow.start():
    mlflow.autolog()

    model = xgboost.train(
            params = best_result,
            dtrain = training_data,
            num_boost_round = 1000,
            evals = [(valid, "validation")],
            early_stopping_rounds = 50
        )
    
# best_params = {key:value for key, value in best_result.items()}